In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Define quantile loss function
def quantile_loss(y_true, y_pred, quantile=0.5):
    e = y_true - y_pred
    return tf.keras.backend.mean(tf.keras.backend.maximum(quantile * e, (quantile - 1) * e), axis=-1)

# Load data
df = pd.read_csv('panama_dataset.csv')
df.dropna(inplace=True)

# Preprocess data
s1 = MinMaxScaler(feature_range=(0,1))
Xs = s1.fit_transform(df)
s2 = MinMaxScaler(feature_range=(0,1))
ys = s2.fit_transform(df[['target_variable']])

window= 5
X=[]
Y=[]
for i in range(window,len(Xs)):
    X.append(Xs[i-window:i,:])
    Y.append(ys[i])
X,Y = np.array(X), np.array(Y)

In [11]:
# Split data into training and testing sets
X_train1, X_test1, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=False)
print('Train set shape', X_train1.shape)
print('testing set shape', X_test1.shape)

Train set shape (33630, 5, 5)
testing set shape (14413, 5, 5)


In [12]:
def hybrid_model(input_shape, quantile=0.5):
    inputs = tf.keras.Input(shape=input_shape)
    
    # First LSTM layer
    lstm_out = tf.keras.layers.LSTM(32, return_sequences=True)(inputs)
    
    # Second LSTM layer
    lstm_out = tf.keras.layers.LSTM(32, return_sequences=True)(lstm_out)

    # Dropout layer
    #dropout = tf.keras.layers.Dropout(0.25)(lstm_out)
    
    # Flatten the output to remove the time dimension
    flattened = tf.keras.layers.Flatten()(lstm_out)

    # Output layer
    quantile_lstm_out = tf.keras.layers.Dense(1)(flattened)
    
    # Quantile LSTM model
    quantile_lstm_model = tf.keras.Model(inputs=inputs, outputs=quantile_lstm_out)
    quantile_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                                loss=lambda y_true, y_pred: quantile_loss(y_true, y_pred, quantile=quantile))
    
    return quantile_lstm_model


In [13]:
model = hybrid_model((X_train1.shape[1], X_train1.shape[2]))
history = model.fit(X_train1, y_train, epochs=50, batch_size=32)

Epoch 1/50
1051/1051 [==============================] - 12s 7ms/step - loss: 0.0186
Epoch 2/50
1051/1051 [==============================] - 8s 8ms/step - loss: 0.0098
Epoch 3/50
1051/1051 [==============================] - 8s 8ms/step - loss: 0.0087
Epoch 4/50
1051/1051 [==============================] - 8s 8ms/step - loss: 0.0083
Epoch 5/50
1051/1051 [==============================] - 8s 7ms/step - loss: 0.0080
Epoch 6/50
1051/1051 [==============================] - 8s 7ms/step - loss: 0.0078
Epoch 7/50
1051/1051 [==============================] - 7s 7ms/step - loss: 0.0075
Epoch 8/50
1051/1051 [==============================] - 8s 8ms/step - loss: 0.0073
Epoch 9/50
1051/1051 [==============================] - 8s 8ms/step - loss: 0.0071
Epoch 10/50
1051/1051 [==============================] - 11s 10ms/step - loss: 0.0071
Epoch 11/50
1051/1051 [==============================] - 12s 12ms/step - loss: 0.0069
Epoch 12/50
1051/1051 [==============================] - 10s 10ms/step - loss: 0

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 5, 5)]            0         
                                                                 
 lstm_2 (LSTM)               (None, 5, 32)             4864      
                                                                 
 lstm_3 (LSTM)               (None, 5, 32)             8320      
                                                                 
 flatten_1 (Flatten)         (None, 160)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 161       
                                                                 
Total params: 13,345
Trainable params: 13,345
Non-trainable params: 0
_________________________________________________________________


In [15]:
y_pred = model.predict(X_test1)

451/451 [==============================] - 3s 3ms/step


In [16]:
y_pred = s2.inverse_transform(y_pred)
y_test = s2.inverse_transform(y_test)

In [17]:
y_test.shape

(14413, 1)

In [18]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)*100
mse = mean_squared_error(y_test, y_pred)
nrmse = (rmse/np.mean(y_pred))*100

print("RMSE:", rmse)
print("MAE:", mae)
print("MAPE:", mape)
print("MSE:", mse)
print("NRMSE:", nrmse)

RMSE: 33.88594972308657
MAE: 23.73479597391054
MAPE: 2.046811459634818
MSE: 1148.2575886355505
NRMSE: 2.8168780705685084
